In [1]:
import os
import time

from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [66]:
#
# ----------------

# step0. uc.Chrome()로 크롤링 할 driver객체 만들기
wd = webdriver.Chrome()
first_tags = ['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업',
            '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
urls = [str(x) for x in range(100,1001, 100)] # IT의 2차분류

arr = []
for url in urls:
    key = first_tags[(int(url)-100)//100]
    wd.get(f'https://www.jobplanet.co.kr/companies?&industry_id={url}')
    time.sleep(3)

    dict, dict2 = {}, {}
    while 1:
        # (1)element's' & (2)section:nth-child(1)~10 -> nth-child(k)를 떼서, section만 남김
        company_infos = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_4')
        company_names = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_3.cominfo > dt > a')
        
        # now = wd.current_url
        flag = False
        for info, name in zip(company_infos, company_names):
            name = name.text
            
            a = info.text.replace(',','').replace('개의 리뷰','').replace('평균','').replace(' 만원','')
            b = a.split('\n') # ['2507', '평점', '2.8', '4172']
            review, avg_rate, avg_salary = int(b[0]), float(b[2]), int(b[3])

            if review == 0:
                flag = True
                continue
            dict[name] = [review, avg_rate, avg_salary]
        # 마지막 페이지까지 클릭하는게 아니라, review많은 순으로 정렬된 상태이므로,
            # review가 0뜨는 순간, 나머지는 다 0임.
            # 물론, 마지막페이지까지 리뷰가 채워져 있는 경우가 있으면 안되겠지만, 확인해보니 마지막은 다 리뷰수 0임
        if flag: 
            break
        # 다음 페이지 넘기기 & elemen't' - elements하면 x
        wd.find_element(By.CSS_SELECTOR, '#listCompanies > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext').click()
        time.sleep(2)
        # (구)코드 - 굳이 이렇게 할 이유가 전혀 없음. 괜히 또 기다릴 여지도 있음
        # if now == wd.current_url:
        #     break
    
    dict2[key] = dict
    arr.append(dict2)
    print(len(dict2[key].keys())) # 누락 데이터 체크용
    
print(arr, len(arr))

1732
5990
969
3151
643
1149
5437
1974
600
1247
[{'서비스업': {'(주)에스씨케이컴퍼니': [4602, 3.3, 3194], '파리바게뜨 (매장)': [4198, 3.4, 3016], '롯데리아 (매장)': [3082, 3.3, 3098], '한국맥도날드(유)': [2936, 3.4, 3218], '(주)트랜스코스모스코리아': [2253, 2.6, 2762], '(주)유베이스': [2102, 2.6, 2722], '(주)하나투어': [1801, 3.0, 3397], '(주)세스코': [1597, 3.0, 3501], '(주)에스텍시스템': [1596, 2.4, 2777], '(주)유니에스': [1529, 2.6, 2758], '(주)아워홈': [1480, 2.7, 4111], '(주)에스케이쉴더스': [1480, 2.6, 3745], '씨제이푸드빌(주)': [1471, 2.9, 3849], '(주)커피빈코리아': [1457, 2.8, 2598], '애슐리 (매장)': [1424, 3.0, 2924], '(주)비케이알': [1419, 3.0, 3068], '(주)호텔신라': [1281, 3.0, 4124], '빕스 (매장)': [1206, 3.4, 2392], '(주)에스원': [1112, 2.8, 3838], '엔제리너스 (매장)': [1074, 3.3, 2570], '(주)삼구아이앤씨': [1046, 2.7, 3613], '롯데월드': [995, 3.4, 4063], '(주)메타넷엠플랫폼': [964, 2.4, 2670], '아웃백스테이크하우스코리아(유)': [947, 3.3, 2876], '이트너스(주)': [931, 2.6, 3435], '코웨이(주)': [926, 3.0, 5333], '뚜레쥬르 (매장)': [897, 3.4, 2776], '던킨/던킨도너츠 (매장)': [881, 3.3, 2782], '더케이텍(주)': [771, 2.4, 3114], '(주)제니엘': [770, 2.8, 2835], '한국마사회'

In [ ]:
# 1732
# 5990
# 969
# 3151
# 643
# 1149
# 5437
# 1974
# 600
# 1247

In [68]:
import pandas as pd

english_first_tags = ['SERVICE', 'MANUFAC_CHEMI', 'MEDICAL', 'DISTRI_TRADE',
            'EDUCATION', 'CONSTRUCTION', 'IT', 'MEDIA', 'FINANCIAL', 'ASSOCIATION']
column_list = ['리뷰수(개)', '평점', '평균 연봉(만)']

# for k in range(len(first_tags)):
for k in range(0,10):
    tag = arr[k][first_tags[k]]

    data_for_df = []
    for key, value in tag.items():
        data_for_df.append(value)

    df = pd.DataFrame(data_for_df, index=tag.keys(), columns=column_list)
    # 0인 값들 빼고, 평균 구해야지 ㅇㅇ. - 문법 이렇게 씀. 까먹,, 다시 기억하기
    average_salary = round(df.loc[df['평균 연봉(만)'] != 0, '평균 연봉(만)'].mean())
    # '평균 연봉(만)' 열에서 0인 값을 찾아 위에서 계산한 평균값으로 대체
    df.loc[df['평균 연봉(만)'] == 0, '평균 연봉(만)'] = average_salary
    
    print(df.tail())

    df.to_csv(f'1-{k}.{english_first_tags[k]}.csv', encoding='utf-8-sig')



            리뷰수(개)   평점  평균 연봉(만)
현대에이엠씨(주)        1  3.0      1374
홍진(주)            1  4.0      4100
효산               1  3.0      1512
후케어스             1  5.0      3158
훼미리마트수택한숲점       1  3.0      3158
             리뷰수(개)   평점  평균 연봉(만)
휴먼브릿지코리아(주)       1  3.0      1755
휴먼텍(주)            1  2.0      2553
휴엘                1  3.0      1755
흥진테크              1  4.0      4100
히트콤(주)            1  5.0      2145
             리뷰수(개)   평점  평균 연봉(만)
하이케어넷(주)          1  1.0      3466
한서대학교재활요양병원       1  3.0      2925
한스치과의원            1  3.0      3900
화타병원              1  3.0      3023
효성정형외과의원          1  3.0      3075
           리뷰수(개)   평점  평균 연봉(만)
화인컴퍼니(주)        1  4.0      4875
효진(주)           1  2.0      3806
훤리스코리아(주)       1  1.0      3477
휴먼리테일           1  3.0      3477
휴먼크로스(주)        1  4.0      2942
                 리뷰수(개)   평점  평균 연봉(만)
푸른학원(주)               1  4.0      3220
하늘교육노량진중앙학원(주)        1  4.0      7175
한동대학교 첨단기계기술연구소       1  4.0      1113
한빛고시학원                